<img src="https://huntingtonarchive.org/Exhibitions/images/fullScreen/COB/01/01_02.jpg" alt="Victory over Mara" width="500"/>

*\"Victory over Mara" (ca. 11th century). Image source: [Circle of Bliss Exhibit](https://huntingtonarchive.org/Exhibitions/circleOfBlissExhibit.php) by the Huntington Archive.* 

##Sutta Meditation v0.0.2

Sutta Meditation creates playlists from [SuttaCentral Voice](https://voice.suttacentral.net/scv/index.html#/sutta) with customizable intervals of silence in-between suttas. Adding these moments of quiet allows for reflection or "sutta meditation". To use this notebook, first run the cell below and then make your own playlist using the documentation and examples!

In [0]:
#@title <--- Please click here!
%%capture

import urllib.request

!pip install pydub
from pydub import AudioSegment


def make_audio(basket="", book="", first_sutta="", last_sutta="", 
               balance=0.25, dist=1.5, initial_silence=0, voice="Raveena",
               export=False):

    if not (book or first_sutta or last_sutta):
        if basket.casefold() == "sn" or not basket:
            basket = "sn"
            book = 3
            first_sutta = 21
            last_sutta = 25
        elif basket.casefold() == "dn":
            first_sutta = 7
            last_sutta = 7
        elif basket.casefold() == "mn":
            first_sutta = 19
            last_sutta = 20
        elif basket.casefold() == "an":
            book = 8
            first_sutta = 1
            last_sutta = 5
        elif basket.casefold() == "thag":
            book = 7
            first_sutta = 1
            last_sutta = 5
        elif basket.casefold() == "thig":
            book = 7
            first_sutta = 1
            last_sutta = 3
    elif not book and basket.casefold() in ["sn", "an", "thig", "thag"]:
        return("Please specify book! (See documentation.)")
    elif basket.casefold() not in ["dn", "mn", "sn", "an", "thag", "thig"]:
        return("Please enter a valid basket! (See documentation.)")
    elif not first_sutta and last_sutta:
        return("Please specify the first sutta!")
    elif not first_sutta and not last_sutta:
        return("Please specify which suttas you'd like to hear!")
    elif book and basket.casefold() in ["dn", "mn"]:
        book = ""
    if first_sutta and not last_sutta:
        last_sutta = first_sutta
    if voice.casefold() == "sujato":
        suffix = "_en"
    else:
        suffix = ""
    if not book:
        dot_or_not = ""
    else:
        dot_or_not = "."
    suttas = list(range(first_sutta, last_sutta + 1))
    nsilences = len(suttas) - 1

    def silence_distribution(n):
        global norm_silence_weights
        if nsilences == 1:
            norm_silence_weights = [1]
        else:
            silence_weights = []
            norm_silence_weights = []
            for i in range(nsilences):
                silence_weights.append(1 + i*(n-1)/(nsilences-1))
            for j in silence_weights:
                norm_silence_weights.append(j/sum(silence_weights))
    silence_distribution(dist)
    print("""Downloading suttas...
          """)

    playlist = []
    for i in range(len(suttas)):
        url = ("https://voice.suttacentral.net/scv/download/playlist/en/"
               + voice + suffix + "/" + basket + str(book) + dot_or_not
               + str(suttas[i]) + "%2Fen%2Fsujato")
        urllib.request.urlretrieve(url, "sutta.mp3")
        sutta = AudioSegment.from_mp3("sutta.mp3")
        playlist.append(sutta)
    print("""Creating playlist...
          """)
    sutta_duration = 0
    for i in range(len(playlist)):
        sutta_duration = sutta_duration + playlist[i].duration_seconds
    silence_duration = [1000*i*balance*sutta_duration
                        for i in norm_silence_weights]
    silencelist = []
    for i in range(nsilences):
        silence = AudioSegment.silent(silence_duration[i])
        silencelist.append(silence)

    mp3 = AudioSegment.silent(initial_silence*60*1000) + playlist[0]
    for i in range(nsilences):
        mp3 = mp3 + silencelist[i] + playlist[i+1]
    if export == True:
        mp3_name = (basket + str(book) + dot_or_not + str(suttas[0]) + "--" 
                    + str(suttas[-1]) + ".mp3")
        mp3.export(str(mp3_name), format="mp3")
        print("Success! See https://stackoverflow.com/a/53860114/11591024 "
              + "for how to download " + "\"" + mp3_name + "\"")
    else:
        return(mp3)

## Documentation

Use the `make_audio()` function to create a playlist with silences in-between the suttas! `make_audio()` has the following options:

* **basket** takes the values "DN", "MN", "SN", "AN", "Thig", or "Thag". (See the section Abbreviations below if you're unsure what these letters mean.) See also https://tinyurl.com/nikayachapters for a table of contents of DN, MN, SN, and AN with links to SuttaCentral. (To be precise, Thig and Thag are not baskets; they are part of the Khuddaka-basket. But we'll treat them as baskets regardless.)
* **book** indicates the SN *saṃyutta* (1-56), AN *nipāta* (1-11), Thig *therīgāthā* (1-16), or Thag *theragāthā* (1-21).
* **first_sutta** is the first sutta to be included in the playlist (e.g. 17)
* **last_sutta** is the last sutta to be included in the playlist (e.g. 22). NOTE: The first and last sutta have to be in the same book!
* **balance** the balance between "silence" and "sutta". Example: if `balance` is set to 0.25 there will be 15 seconds of silence for every minute of sutta on average. (`balance` uses the actual duration of the selected suttas.)
* **dist** controls how the silence is distributed throughout the playlist. For example, if set to 1, all silences will be equally long. If set to 1.5, the silence durations will increase linearly so that the last silence will be 1.5x longer than the first silence. Note that the total amount of silence will remain the same; `dist` only changes how the silence is distributed!
* **initial_silence** how much silence to include before the first sutta (in minutes). Set to 0 if you want to start directly with the first sutta, or (for example) to 1/6 if you first want 10 seconds of silence.
* **voice** "Amy", "Russell", "Raveena" (default), "Matthew" or "Sujato": see the SCV settings and documentation.
* **export** if set to `True`, your file will be given a name and saved as an mp3 in Google Colab. See https://stackoverflow.com/a/53860114/11591024 for how to download files saved in Google Colab.

By default, `make_audio()` will make the Kosala chapter (SN 3.21--25).

* **make_audio(basket="sn", book=3, first_sutta=21, last_sutta=25, balance=0.25, dist=2 initial_silence=0, voice="raveena", export=0)**


## Bugs & Limitations

Sutta Meditation has a few known bugs and shortcomings.

* **Please be gentle with how many suttas you include, especially if they are long suttas.** Sutta Meditation has trouble handling mp3s exceeding 1½ hours.
* Some suttas (such as https://suttacentral.net/an1.1-10) are baked into a chapter. For now, Sutta Meditation cannot handle them properly, so they're best avoided. Luckily, this affects only a few suttas.
* For now, only consecutive suttas in the same book can be processed. 
* Pali is not yet supported.
* There seems to be some loss of audio quality.

## *Examples*

* The Kosala chapter

In [0]:
# Listen to the Kosala chapter!
make_audio() 

* The first five verses of the Therīgātha.

In [0]:
# Five verses by senior nuns.
make_audio("thig")

* SN 1.1 and SN 1.2 by Bhante Sujato

In [0]:
# Two discourses narrated by Bhante Sujato.
make_audio(basket="sn", book=1, first_sutta=1, last_sutta=2, voice="sujato")

Try making your own playlist by adjusting the options in the cell above!

## Abbreviations

### DN

> The Long Discourses (Dīgha Nikāya, abbreviated DN) is a collection of 34 discourses in the Pali canon (Tipiṭaka) of the Theravāda school. The word “long” refers to the length of the individual discourses, not the collection as a whole, which is in fact the smallest of the five Pali Nikāyas. It is one of the fundamental collections of early Buddhist teachings, depicting the Buddha in a lively range of settings. Compared to other collections it contains more extended narratives in diverse literary styles. Many discourses feature interreligious dialog with brahmins and other non-Buddhists. This collection parallels the Dīrghāgama (DA) of the Dharmaguptaka school, which is the first text in the Taishō edition of the Chinese canon. Several uncollected suttas in Chinese and Sanskrit also belong to this collection. Two-thirds of a Dīrghāgama from the Sarvāstivāda school has been found, but only small portions have been published.
>
> Source: https://suttacentral.net/dn

### MN

> The Middle Discourses (Majjhima Nikāya, abbreviated MN) is a collection of 152 discourses in the Pali canon (Tipiṭaka) of the Theravada school of Buddhism. The word “middle” refers to the length of the individual discourses. This is perhaps the most popular collection of early discourses. It contains a wide variety of teachings, many of them presented as narratives between the Buddha and a diverse range of his contemporaries. The collection parallels the Madhyamāgama (MA) of the Sarvāstivāda school, which survives as a translation in the Chinese canon.
>
> Source: https://suttacentral.net/mn

### SN

> The “Linked” or “Connected” Discourses (Saṁyutta Nikāya, abbreviated SN) is a collection of over a thousand short discourses in the Pali canon. The word “linked” refers to the fact that the texts are collected and organized by topic. In most cases the organizing principle is a particular theme of Dhamma, for example, the five aggregates, dependent origination, the noble eightfold path, mindfulness meditation, or the four noble truths. This collection contains the most extensive range of texts on these core themes. In other cases chapters are organized according to the person or kind of person who speaks. This collection has a full parallel in the Saṁyuktāgama (SA) of the Sarvāstivāda school in Chinese translation. In addition, there are two partial collections in Chinese (SA-2 and SA-3) as well as a number of miscellaneous or fragmentary texts in Chinese, Sanskrit, and Tibetan. Much of the organizational structure of SN is shared with SA, suggesting that this structure preceded the split between these two collections.
>
> Source: https://suttacentral.net/sn

### AN

> The “Numbered” or “Numerical” Discourses are usually known as Aṅguttara Nikāya in Pali, abbreviated AN. However, the Pali tradition also knows the form Ekottara (“one-up” or “incremental”), and this is the form usually found in the northern collections. These collections organize texts in numbered sets, from one to eleven. Compared to the other nikāyas, they are more oriented to the lay community. The Ekottarikāgama (EA) in Chinese is a highly unusual text, which features a range of variations within itself when it comes even to basic doctrines. It shares considerably less in common with the Pali Aṅguttara than the other collections do with their counterparts. In addition, there is a partial Ekottarikāgama in Chinese, as well as a variety of individual discourses and fragments in Chinese and Sanskrit.
>
> Source: https://suttacentral.net/an

###Thag 

> The “Verses of the Senior Monks” is a collection of about 1288 verses attributed to 264 of the senior monks alive in the Buddha’s time, or in a few cases, a little later. It is a pair with the Therīgāthā, the “Verses of the Senior Nuns”. These verses celebrate the joy of freedom and the life of meditation in the forest. Together these collections constitute one of the oldest and largest collections of contemplative literature, preserving the unique voices of hundreds of early practitioners. Based on style and content, these collections belong to the early discourses. They are referred to on occasion in the northern canons, but no parallel collections have survived.
>
> Source: https://suttacentral.net/thag

###Thig

> The “Verses of the Senior Nuns” is a collection of about 524 verses attributed to 73 of the senior nuns alive in the Buddha’s time, or in a few cases, a little later. These verses celebrate the bliss of freedom and the life of meditation, full of proud and joyous proclamations of their spiritual attainments and their gratitude to other nuns as guides and teachers. The Therīgāthā is one of the oldest spiritual texts recording only women’s voices. It is a pair with the Theragāthā, the “Verses of the Senior Monks”. Together these collections constitute one of the oldest and largest collections of contemplative literature. Based on style and content, these collections belong to the early discourses. They are referred to on occasion in the northern canons, but no parallel collections have survived.
>
> Source: https://suttacentral.net/thig

## v0.0.2 changes

Case-insensitive input, defaults for all six baskets, code adjusted to PEP 8, feedback for incomplete input.